# BEAM SEARCH
- Pour une couche, tire n character et stocker les plus propables

# LSTM
- genere les parantheses (automates) pour montrer que rnn va les oublie mais LSTM non

In [74]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

from torch import nn
import torch
import glob

from sklearn.model_selection import train_test_split
from tools import AverageMeter

from charDataset import CharDataset
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size):
        super().__init__()
        c_size = input_size + hidden_size
        self.C = torch.zeros(hidden_size,1)
        self.f_t = nn.Linear(in_features=c_size, out_features=hidden_size, bias=True)
        self.sigmoid = nn.Sigmoid()
        self.f_t2 = nn.Linear(in_features=c_size, out_features=hidden_size, bias=True)
        self.C_t = nn.Linear(in_features=c_size, out_features=hidden_size, bias=True)
        self.tanh = nn.Tanh()
        self.out = nn.Linear(in_features=c_size, out_features=hidden_size, bias=True)
        
    def forward(self, h, x):
        combined = torch.cat((h,x), dim=0).t()
        
        # CLEAR
        f_t = self.sigmoid(self.f_t(combined)).t()
        self.C = self.C * f_t
        print(self.C.shape)
        
        # WRITE
        i_t = (self.sigmoid(self.f_t2(combined)) * self.tanh(self.C_t(combined))).t()
        self.C = self.C + i_t
        print(self.C.shape)
        
        # READ
        out = self.sigmoid(self.out(combined)).t()
        h_ = self.C * out
        print(self.C.shape)
        return h_

In [58]:
input_size = 9
hidden_size = 455
lstm = LSTM(input_size,hidden_size)
x = torch.zeros(input_size,1)
h = torch.zeros(hidden_size,1)
for i in range(5):
    h = lstm(h,x)

torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])
torch.Size([455, 1])


In [77]:
cdataset = CharDataset('train_data.tx', 'vocab.tx', 10)
dataload = DataLoader(cdataset,batch_size=100,shuffle=True)

cutting off end of data so that the batches/sequences divide evenly


In [81]:
for (batch, target) in dataload:
    print(target.shape)
    break

torch.Size([100, 10])
